In [9]:
import pandas as pd
from pathlib import Path
import datetime

pasta_origem = Path("bases_C")
arquivo_saida = Path("OUTPUT") / "Acompanhamento.xlsx"
arquivo_saida.parent.mkdir(exist_ok=True)

colunas_uteis = [
    'FRANQUEADO',
    'RESSUPRIR',
    'VELOCIDADE_VENDA',
    'ALVO',
    'DATA',
    'FILIAL'
]

abas_alvo = [
    "Base rep.", 
    "Base Rep.", 
    "Base Rep", 
    "Base rep", 
    "Resultado Validacao", 
    "Resultado Validação"
]


lista_dfs = []

arquivos = list(pasta_origem.glob("*.xlsx")) # Procura por todos os arquivos que terminam com ".xlsx" dentro da pasta "bases_C" e cria uma lista com eles.
print(f"encontramos {len(arquivos)} arquivos. Iniciando leitura turbinada com Calamine")

for arquivo in arquivos:
    try:
        
        xls = pd.ExcelFile(arquivo, engine="calamine") #esse comando apenas "abre a conexão" com o arquivo Excel.
        abas_existentes = xls.sheet_names #devolve uma lista com os nomes de todas as abas que existem naquele arquivo.
        aba_para_ler = None

        for aba_alvo in abas_alvo:
            if aba_alvo in abas_existentes:
                aba_para_ler = aba_alvo
                break

        if aba_para_ler is None:
            print(f"ALERTA: nenhuma aba válida encontrada em {arquivo.name}. Abas disponíveis : {abas_existentes}")
            continue # Pula para o próximo arquivo se não achou a aba

        df_temp = pd.read_excel(xls, sheet_name=aba_para_ler, engine="calamine")


        df_temp.columns = df_temp.columns.str.upper().str.strip()
        

        colunas_presentes = [col for col in colunas_uteis if col in df_temp.columns]

        df_temp = df_temp[colunas_presentes].copy() # O ".copy()" evita avisos de alteração de dados em uma cópia.

        if 'DATA' in df_temp.columns:
            df_temp['DATA'] = pd.to_datetime(df_temp['DATA'], errors="coerce") # Converte a coluna 'DATA' para o formato de data. Se algum valor não for uma data válida, ele se tornará "NaT" (Not a Time).

        lista_dfs.append(df_temp) # Converte a coluna 'DATA' para o formato de data. Se algum valor não for uma data válida, ele se tornará "NaT" (Not a Time).
        print(f"Lido: {arquivo.name} - {len(df_temp)} linhas")
    except Exception as e: # Informa qual arquivo deu erro e qual foi a mensagem de erro.
        print(f"Erro ao ler {arquivo.name}: {e}")

if not lista_dfs:
    print("Nenhum arquivo foi lido corretamente. Verifique a pasta.")
    exit()


print("Empilhando dataframes...")

df_geral = pd.concat(lista_dfs, ignore_index=True)
linhas_antes = len(df_geral)
df_geral = df_geral.dropna(subset=['DATA'])
print(f"Linhas com data nula removidas: {linhas_antes - len(df_geral)}")
df_geral['DATA'] = df_geral['DATA'].dt.strftime('%d/%m/%Y')
print(f"base total consolidado {len(df_geral)} linhas. iniciando criaçãodas abas...")

encontramos 12 arquivos. Iniciando leitura turbinada com Calamine
Lido: (Validação) 27ª rodada_29.09.xlsx - 372158 linhas
Lido: (Validação) 28ª rodada_07.10.xlsx - 373365 linhas
Lido: (Validação) 29ª rodada_14.10.xlsx - 369462 linhas
Lido: (Validação) 30ª rodada_21.10.xlsx - 375271 linhas
Lido: (Validação) 31ª rodada_28.10.xlsx - 358627 linhas
Lido: (Validação) 32ª rodada_06.11.xlsx - 337455 linhas
Lido: (Validação) 33ª rodada_12.11.xlsx - 343244 linhas
Lido: (Validação) 34ª rodada_19.11.xlsx - 322810 linhas
Lido: (Validação) 35ª rodada_24.11.xlsx - 353299 linhas
Lido: (Validação) 36ª rodada_02.12.xlsx - 356977 linhas
Lido: (Validação) 37ª rodada_08.12.xlsx - 18790 linhas
Lido: (Validação) 38ª rodada_17.12.xlsx - 4720 linhas
Empilhando dataframes...
Linhas com data nula removidas: 0
base total consolidado 3586178 linhas. iniciando criaçãodas abas...


In [10]:
colunas_ordenadas = sorted(df_geral['DATA'].unique(), key=lambda x: pd.to_datetime(x, dayfirst=True)) #Pegue as datas únicas da coluna DATA. Para saber a ordem certa, converta temporariamente cada uma para o formato de data real (datetime) usando o padrão 'dia primeiro' (dayfirst), e ordene do mais antigo para o mais novo"
colunas_ordenadas = [pd.to_datetime(d, dayfirst=True).strftime('%d/%m/%Y') for d in colunas_ordenadas] # Esta segunda linha percorre a lista de datas já ordenadas e garante que TODAS estejam no formato de texto "dd/mm/aaaa". É uma etapa de padronização.

In [11]:
def criar_pivot(df, coluna_valor, ordem_colunas):
    pivot = pd.pivot_table(df, index=['FRANQUEADO', 'FILIAL'], columns='DATA', values=coluna_valor, aggfunc="sum", fill_value=0)
    pivot = pivot.reindex(columns=ordem_colunas, fill_value=0)
    pivot = pivot.reset_index()

    return pivot

In [12]:
df_vdv = criar_pivot(df_geral, 'VELOCIDADE_VENDA', colunas_ordenadas)
df_ressuprir = criar_pivot(df_geral, 'RESSUPRIR', colunas_ordenadas)
df_alvo = criar_pivot(df_geral, 'ALVO', colunas_ordenadas)

for df in [df_vdv, df_ressuprir, df_alvo]:
        df['FILIAL'] = df['FILIAL'].astype(int)

print(f"salvando em : {arquivo_saida}")

with pd.ExcelWriter(arquivo_saida, engine='xlsxwriter') as writer:
    df_vdv.to_excel(writer, sheet_name="VDV", index=False)
    df_ressuprir.to_excel(writer, sheet_name="RESSUPRIR", index=False)
    df_alvo.to_excel(writer, sheet_name="ALVO", index=False)

    workbook = writer.book
    formato_inteiro = workbook.add_format({'num_format':'#,##0'})

    for aba in ['VDV', 'RESSUPRIR', 'ALVO']:
        worksheet = writer.sheets[aba]

        worksheet.set_column(0, 0, 40) # Coluna A (Franqueado) Larga
        worksheet.set_column(1, 1, 10) ## Coluna B (Filial)
        ultima_coluna = 1 + len(colunas_ordenadas)

        if aba == 'VDV':
             worksheet.set_column(2, ultima_coluna, 12)
        else:
             worksheet.set_column(2, ultima_coluna, 12, formato_inteiro)
        

print("Sucesso! Relatório gerado.")



    

salvando em : OUTPUT\Acompanhamento.xlsx
Sucesso! Relatório gerado.
